In [ ]:
import numpy as np
import sys
import csv
import random
import copy
import string
import time
import datetime
import itertools
from tqdm import tqdm
from scipy.stats import pareto
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
rng = np.random.default_rng(int(time.time()))

# 1. Helper Functions

In [ ]:
##########################################
#### Functions for plotting Functions  ###
##########################################
exec(open('../matplot-lib-utils.py').read())

In [ ]:
##################################################
####### Fetch: General Helper Functions  #########
##################################################
exec(open('../utils.py').read())

In [ ]:
##########################################
### Fetch: Algorithms and Baseline ######
##########################################
exec(open('../algorithms.py').read())

In [ ]:
####################################################
### Fetch: Helper function for the simulation ######
####################################################
exec(open('simulation-utils.py').read())

In [ ]:
DEBUG = False
def print_debug(s):
    if DEBUG: print(s)

# 2. Parse Data

In [ ]:
genres = set()
movie_ids = []
movie_id_to_index = {}
user_ids = []
movie_names = {}

with open('ml-20m/movies.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    i = 0
    
    for row in csv_reader:
        if i == 0: 
            i += 1
            continue 
            
        movie_ids.append(int(row[0]))
        movie_id_to_index[int(row[0])] = i-1
        movie_names[i-1] = row[1]
        
        tmp_genres = row[2].split('|')
        for g in tmp_genres:
            genres.add(g)
            
        i += 1

genres = sorted(list(genres))

movie_genres = np.zeros((len(movie_ids), len(genres)))

with open('ml-20m/movies.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    i = 0
    
    for row in csv_reader:
        if i == 0: 
            i += 1
            continue 
         
        tmp_genres = row[2].split('|')
        for g1 in tmp_genres:
            for j, g2 in enumerate(genres):
                if g1 == g2:
                    movie_genres[i-1][j] = 1
                    
        i += 1

In [ ]:
print(f'There are {len(movie_ids)} movies')
print(f'There are {len(genres)} unique genres')
print(f'On average there are {np.sum(movie_genres) / len(movie_ids)} genres per movie')

print('')

for j in range(10):
    print(f'{np.round(np.mean(np.sum(movie_genres, axis=1)==j)*100, 2)}%\tof movies have {j} genre(s)')
    
print('')

print(f'Maximum number of genres of a movie are {np.max(np.sum(movie_genres, axis=1))}')

## 2.1. Fetch Relevance Scores

In [ ]:
# convert genre names to lower case (to match the names of 'tags')
for i in range(len(genres)):
    genres[i] = genres[i].lower()

In [ ]:
tags = []


with open('ml-20m/genome-tags.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    i = 0
    
    for row in tqdm(csv_reader):
        if i == 0: 
            i += 1
            continue 
            
        tags.append(row[1])
            
        i += 1


rev_tags = {} # tag name to index mapping
for i, t in enumerate(tags):
    rev_tags[t] = i
    
cnt = 0 # count number of genres not in tags 
for g in genres:
    if g not in tags:
        cnt += 1
        
print(f'{cnt} genre(s) are absent from tags')
assert(cnt == 3)

movie_tag_rel_scores = {}
movie_ind_with_scores = set()

with open('ml-20m/genome-scores.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    i = 0
    
    for row in tqdm(csv_reader):
        if i == 0: 
            i += 1
            continue
        
        mov_id = movie_id_to_index[int(row[0])]
        tag_id = int(row[1])-1
        movie_ind_with_scores.add(mov_id)
        score = float(row[2])
        
        if tags[tag_id] not in genres: continue 
            
        if mov_id not in movie_tag_rel_scores:
            movie_tag_rel_scores[mov_id] = {}
        
        movie_tag_rel_scores[mov_id][tag_id] = score
            
        i += 1


In [ ]:
print(f'{np.round(len(movie_ind_with_scores)/len(movie_ids)*100, 2)}% of the movies have relevance scores')

## 2.2. Fetch User Ratings

In [ ]:
user_ratings = {} # userid x (list of dictionaries describing each rating given by this user_id)
movie_rating_cnt = {} # movie index x number of total ratings given to this movie index
movie_rating_sum = {} # movie index x sum of total ratings given to this movie index

In [ ]:
with open('ml-20m/ratings.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    i = 0
    
    for row in tqdm(csv_reader):
        if i == 0: 
            i += 1
            continue 
            
        user_id = int(row[0]) 
        mov_ind = movie_id_to_index[int(row[1])]
        rating = float(row[2])
        
        if user_id not in user_ratings:
            user_ratings[user_id] = []
        if mov_ind not in movie_rating_cnt:
            movie_rating_cnt[mov_ind] = 0
        if mov_ind not in movie_rating_sum:
            movie_rating_sum[mov_ind] = 0

        user_ratings[user_id].append({'movie_id': int(row[1]), 'movie_index': mov_ind, 'rating': rating})
        movie_rating_cnt[mov_ind] += 1
        movie_rating_sum[mov_ind] += rating
        
        i += 1

## 3. Fetch name of lead actor

In [ ]:
import os
import pickle
import requests
from bs4 import BeautifulSoup
from joblib import Parallel, delayed

In [ ]:
missing = 0
parsing_error = 0
parsing_errors = []
movie_cast = {}
cnt_exception = 0

if False:
    for i in tqdm(movie_ids):
        if not os.path.isfile('scraped_pages/'+str(i)+'.html'): 
            missing += 1
            continue

        try:
            f = open('scraped_pages/'+str(i)+'.html', "r")
            soup = BeautifulSoup(f.read(), "html.parser")
            results = soup.find(id = "main-container")

            data_elements = results.find_all("div", class_="heading-and-data")

            cast = data_elements[-2].find_all("span")

            movie_cast[i] = []

            for c in cast:
                movie_cast[i].append(c.find_all("a")[0].text)
        except:
            print(f'Error in parsing movie {i}')
            parsing_errors.append(i)

            cnt_exception += 1
            
    
    filehandler = open(f"movie_cast"+file_str()+rand_string(5)+".obj","wb")
    pickle.dump(movie_cast, filehandler)
    filehandler.close()


In [ ]:
filehandler = open(f"movie_castH00M17S52_05-13-22gdipj.obj","rb")
movie_cast = pickle.load(filehandler) # a dictionary: movie_id x a list of cast members' names [movie id is changed to movie index below]
filehandler.close()

In [ ]:
movie_cast_ind = {}

for k in movie_cast.keys():
    movie_cast_ind[movie_id_to_index[k]] = copy.deepcopy(movie_cast[k])

In [ ]:
movie_cast = movie_cast_ind # a dictionary: movie index x a list of cast members' names

## 4. Predict (binary) genders of lead actors

In [ ]:
run_genderize = False
if run_genderize:
    # Note: There are 27k movies in the dataset. We will call Genderize once for every movie
    # This makes 27k calls to Genderize which exceeds the 1k free API calls offered by Genderize
    # We provide a pickled file with responses from Gendrize
    # But if you would like to directly call the Genderize API pleas enter an API key below
    from genderize import Genderize
    
    API_KEY = ""
    genderize = Genderize(
    user_agent='GenderizeDocs/0.0',
    api_key=API_KEY,
    timeout=5.0)

In [ ]:
movie_ind_present = list(movie_cast.keys()) 

In [ ]:
genres_matching_tags = copy.deepcopy(genres)
genres_matching_tags.remove('film-noir')
genres_matching_tags.remove('imax')
genres_matching_tags.remove('(no genres listed)')
genres_matching_tags = sorted(genres_matching_tags)

In [ ]:
to_delete = []
for ind in tqdm(movie_ind_present):
    cast = movie_cast[ind]
    if len(cast) == 0:
        to_delete.append(ind)

for m in to_delete:
    movie_ind_present.remove(m)

In [ ]:
li = []
cnt = 0
j = 0
for ind in tqdm(movie_ind_present):
    if cnt % 100 == 0:
        li.append([])
        j += 1
        
    cast = movie_cast[ind]
    if len(cast) == 0:
        print('oops')
        continue 
    lead = cast[0]
    name = lead.split(' ')
    first_name = name[0]
    li[j-1].append((ind, first_name))
    cnt += 1

predictions_all = []

############################
### set `run_genderize=True` to call Genderize API (not necessary to execute the rest of the code)
############################
if run_genderize:
    li = []
    cnt = 0
    j = 0
    for i in tqdm(movie_ids_present):
        if cnt % 100 == 0:
            li.append([])
            j += 1

        cast = movie_cast[i]
        if len(cast) == 0:
            continue 
        lead = cast[0]
        name = lead.split(' ')
        first_name = name[0]
        li[j-1].append((i, first_name))
        cnt += 1

    predictions_all = []

    for jj in tqdm(range(j)):
        names = [ll[1] for ll in li[jj]]

        predictions = genderize.get(names)
        predictions_all.extend(predictions)

    filehandler = open(f"predictions_all"+file_str()+rand_string(5)+".obj","wb")
    pickle.dump(predictions_all, filehandler)
    filehandler.close()

In [ ]:
# load pickled responses from Genderize
if not run_genderize:
    filehandler = open(f"predictions_allH00M17S50_05-13-22liism.obj","rb")
    predictions_all = pickle.load(filehandler) 
    filehandler.close()

In [ ]:
assert(len(predictions_all) == len(movie_ind_present))

In [ ]:
for i in range(len(predictions_all)):
    
    if movie_cast[movie_ind_present[i]][0].split(' ')[0] != predictions_all[i]['name']:
        print(i)
        print(predictions_all[i])
        print(movie_cast[movie_ind_present[i]][0])
        assert(False)
    
    # print(movie_cast[movie_ind_present[i]])

In [ ]:
grps = [[], []]
threshold_prob = 0.1 ### set this threshold to 0.5 to compute stats for Table 3 in the appendix
threshold_count = 100
fail_cnt = 0

for i, ind in tqdm(enumerate(movie_ind_present)):
    prediction = predictions_all[i]
    
    if ind not in movie_ind_with_scores:
        continue 

    pred_female = -1
    if prediction['gender'] == 'female':
        pred_female = prediction['probability']
    elif prediction['gender'] == 'male':
        pred_female = 1-prediction['probability']
    else:
        fail_cnt += 1
        continue 

    if pred_female > 1 - threshold_prob:
        grps[1].append(ind)
    elif pred_female <= threshold_prob:
        grps[0].append(ind)
    else:
        continue 

print(f'Genderize predicts the gender of the lead actor, with confidence',
  f'{1-threshold_prob} or higher for',
  f'{np.round((len(grps[0])+len(grps[1]))/len(movie_ind_with_scores), 2)} fraction of the movies (with scores)')

In [ ]:
print('number of male-led movies, female-led movies, and fraction of movies led by females')
len(grps[0]), len(grps[1]), len(grps[1])/(len(grps[1])+len(grps[0]))

### The data in the cell right below is not reported in the paper

In [ ]:
####################################################################################
####################################################################################
### AVERAGE RELEVANCE SCORES OF FEMALE-LED AND MALE-LED MOVIES IN DIFFERENT GENRES
####################################################################################
####################################################################################
grp_avgs = [np.zeros(len(genres_matching_tags)), np.zeros(len(genres_matching_tags))]

fg=1
cnt=0
missing = {}
for l in [0, 1]:
    for ind in grps[l]:
        for i, g in enumerate(genres_matching_tags):
            if ind not in movie_tag_rel_scores:
                missing[ind]=0
                print(ind)
                assert(False)
            else:
                grp_avgs[l][i] += movie_tag_rel_scores[ind][rev_tags[g]]

    grp_avgs[l] /= len(grps[l])
    
print('AVERAGE RELEVANCE SCORES OF FEMALE-LED AND MALE-LED MOVIES IN DIFFERENT GENRES')
print('')
print(f"Genre{' '*(10-len('Genre'))}\tMale-led\tFemale-led\tRatio (female-led/male-led)")
print('-'*74)
for i, g in enumerate(genres_matching_tags):
    print(f"{g}{' '*(10-len(g))} \t{np.round(grp_avgs[0][i], 4)}\t\t{np.round(grp_avgs[1][i], 4)}\t\t{np.round(grp_avgs[1][i]/grp_avgs[0][i],4)}")
print('-'*74)
print(f"{'Total'}{' '*(10-len('Total'))} \t{np.round(np.sum(grp_avgs[0]), 4)}\t\t{np.round(np.sum(grp_avgs[1]), 4)}\t\t{np.round(np.sum(grp_avgs[1])/np.sum(grp_avgs[0]),4)}")    
print('='*74)

### Table 1 in paper

In [ ]:
####################################################################################
####################################################################################
### AVERAGE RELEVANCE SCORES OF FEMALE-LED AND MALE-LED MOVIES IN DIFFERENT GENRES (COMPUTED ONLY OVER MOVIES IN THE SPECIFIC GENRE)
####################################################################################
####################################################################################

grp_avgs_conditioned_on_genre = [np.zeros(len(genres_matching_tags)), np.zeros(len(genres_matching_tags))]

mp_genre_to_ind = {}
for i, g in enumerate(genres):
    mp_genre_to_ind[g] = i

missing = {}
for l in [0, 1]:
    cnt = 0
    for ind in tqdm(grps[l]):
        for i, g in enumerate(genres_matching_tags):
            g_ind = mp_genre_to_ind[g]
            
            if movie_genres[ind][g_ind] == 0: continue 
                
            cnt += 1
            
            if ind not in movie_tag_rel_scores:
                missing[ind]=0
                print(ind)
                assert(False)
            else:
                grp_avgs_conditioned_on_genre[l][i] += movie_tag_rel_scores[ind][rev_tags[g]]


    grp_avgs_conditioned_on_genre[l] /= cnt 
    
print('AVERAGE RELEVANCE SCORES OF FEMALE-LED AND MALE-LED MOVIES IN DIFFERENT GENRES (COMPUTED OVER MOVIES IN THIS GENRE)')
print('')
print(f"Genre{' '*(10-len('Genre'))}\tMale-led\tFemale-led\tRatio (female-led/male-led)")
print('-'*74)
for i, g in enumerate(genres_matching_tags):
    print(f"{g}{' '*(10-len(g))} \t{np.round(grp_avgs_conditioned_on_genre[0][i], 4)}\t\t{np.round(grp_avgs_conditioned_on_genre[1][i], 4)}\t\t{np.round(grp_avgs_conditioned_on_genre[1][i]/grp_avgs_conditioned_on_genre[0][i],4)}")
print('-'*74)
print(f"{'Total'}{' '*(10-len('Totalc'))} \t{np.round(np.sum(grp_avgs_conditioned_on_genre[0]), 4)}\t\t{np.round(np.sum(grp_avgs_conditioned_on_genre[1]), 4)}\t\t{np.round(np.sum(grp_avgs_conditioned_on_genre[1])/np.sum(grp_avgs_conditioned_on_genre[0]),4)}")    
print('='*74)

In [ ]:
stereotypically_men_genre = []

i = -1
for j, g in enumerate(genres):
    if g not in genres_matching_tags:
        continue
    i += 1
    ratio = grp_avgs_conditioned_on_genre[1][i] / grp_avgs_conditioned_on_genre[0][i]
    
    if ratio <= 0.5:
        stereotypically_men_genre.append(j)

### Table 2 in paper

In [ ]:
####################################################################################
####################################################################################
### AVERAGE USER RATINGS OF FEMALE-LED AND MALE-LED MOVIES IN DIFFERENT GENRES (COMPUTED ONLY OVER MOVIES IN THE SPECIFIC GENRE)
####################################################################################
####################################################################################


grp_avg_ratings_conditioned_on_genre = [[[] for i in genres_matching_tags], [[] for i in genres_matching_tags]]

for l in [0, 1]:
    cnt = np.zeros(len(genres_matching_tags))
    for ind in tqdm(grps[l]):
        for i, g in enumerate(genres_matching_tags):
            
            g_ind = mp_genre_to_ind[g]
            
            if movie_genres[ind][g_ind] == 0: continue
            
            if ind not in movie_ind_with_scores:
                print(ind)
                assert(False)
        
            if ind not in movie_rating_sum: continue
            if ind not in movie_rating_cnt: continue
            cnt[i] += 1
            assert(movie_rating_sum[ind] / movie_rating_cnt[ind]<=5)
            grp_avg_ratings_conditioned_on_genre[l][i].append(movie_rating_sum[ind] / movie_rating_cnt[ind])
            
            
np.set_printoptions(precision=2)
for i in range(len(genres_matching_tags)):
    print(f'Genre: {genres_matching_tags[i]}{" "*(10-len(genres_matching_tags[i]))}\t'+
          f'Ratio: {np.mean(grp_avg_ratings_conditioned_on_genre[1][i])/np.mean(grp_avg_ratings_conditioned_on_genre[0][i]):.2f}')
    print(f'\t{" "*(10)}\t'+
          f'Averages: {np.mean(grp_avg_ratings_conditioned_on_genre[0][i]):.2f} ({np.std(grp_avg_ratings_conditioned_on_genre[0][i]):.2f}),\t'+
          f'{np.mean(grp_avg_ratings_conditioned_on_genre[1][i]):.2f} ({np.std(grp_avg_ratings_conditioned_on_genre[1][i]):.2f})')

In [ ]:
#############################
#### Compute number of ratings submitted by each user 
#############################
num_ratings_per_user = []

for u in tqdm(user_ratings.keys()):
    num_ratings_per_user.append(0)
    
    for r in user_ratings[u]:
        if r['movie_index'] in movie_ind_with_scores:
            num_ratings_per_user[-1] += 1 
            
num_ratings_per_user = np.array(num_ratings_per_user)
_ = plt.hist(np.clip(num_ratings_per_user, 0, 200), density=True, bins=30)


In [ ]:
user_ratings_for_movies_with_scores = {}

cnt=0
for u in tqdm(user_ratings):
    if u not in user_ratings_for_movies_with_scores:
        user_ratings_for_movies_with_scores[u] = []
        
    for r in user_ratings[u]:
        if r['movie_index'] in movie_ind_with_scores:
            user_ratings_for_movies_with_scores[u].append(r)
        else: cnt+=1
            
print(cnt)

In [ ]:
threshold_on_user_rating_cnt = 200

selected_users = []

for u in tqdm(user_ratings_for_movies_with_scores.keys()):
    if len(user_ratings_for_movies_with_scores[u]) >= threshold_on_user_rating_cnt:
        selected_users.append(u)

selected_users_set = set(selected_users)

In [ ]:
print(f'{len(selected_users) / 138493 * 100}% of the users submitted at least 200 ratings')

In [ ]:
#################################################################
###### Update cnt/sum of movie ratings to only consider ratings from users in `selected_users_set`
#################################################################

movie_rating_cnt_selected = {}
movie_rating_sum_selected = {}

with open('ml-20m/ratings.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    i = 0
    
    for row in tqdm(csv_reader):
        if i == 0: 
            i += 1
            continue 
            
        user_id = int(row[0])
        mov_ind = movie_id_to_index[int(row[1])]
        rating = float(row[2])
        # timestamp = int(row[3])
        
        if mov_ind not in movie_ind_with_scores:
            continue 
        
        if user_id not in selected_users_set: continue 
        
        if mov_ind not in movie_rating_cnt_selected:
            movie_rating_cnt_selected[mov_ind] = 0
        if mov_ind not in movie_rating_sum_selected:
            movie_rating_sum_selected[mov_ind] = 0

        movie_rating_cnt_selected[mov_ind] += 1
        movie_rating_sum_selected[mov_ind] += rating
        
        i += 1

## Simulation

In [ ]:
grps_set = [set(grps[0]), set(grps[1])]


#####################################
### parameter of the submodular function
### They specify the function with g1(x)=g1(x)=...=gm(x)=np.sqrt()
#####################################
func = lambda x: np.sqrt(x)
func2 = lambda x: np.sqrt(x)
weight_F = np.ones(20) # parameter of the submodular function

In [ ]:
def run_simulation(stereotypically_men_genre, num_of_genres=1):

    ITERS = 100

    assert(num_of_genres <= len(stereotypically_men_genre))
    fg=0

    iterator = list(itertools.combinations(stereotypically_men_genre, num_of_genres))
    for gen in tqdm(iterator):
        weight_F = np.ones(20)

        LIST_baseline_uncons_OUTER = []
        LIST_alg_disj_OUTER = []
        ERR_LIST_baseline_uncons_OUTER = []
        ERR_LIST_alg_disj_OUTER = []

        x = []

        print(f'Genre: {[genres[g] for g in gen]}')

        for k in [50, 100, 150, 200]: 
            list_baseline_uncons = []
            list_alg_disj = []

            user_draws = rng.choice(len(selected_users), 10*ITERS)

            ijk = -1
            while len(list_alg_disj) < ITERS and ijk < 10*ITERS:
                ijk += 1
                u_id = selected_users[user_draws[ijk]]
                m_inds = get_rated_movie_indices(u_id)

                m_ind_to_i = {}
                for i, ind in enumerate(m_inds): m_ind_to_i[ind] = i

                n = len(m_inds)
                m = len(genres_matching_tags)

                if n < k: continue 

                obs_util = np.zeros((n, m))

                # Get relevance scores (obbserved utilities)
                for i in range(n):
                    for j in range(m):
                        m_ind = m_inds[i]
                        g = rev_tags[genres_matching_tags[j]] # name of j-th genre 

                        obs_util[i][j] = movie_tag_rel_scores[m_ind][g]

                # Create protected groups
                grps_int = [[], []]
                for i in range(n):
                    m_ind = m_inds[i]

                    if m_ind in grps_set[0]:
                        grps_int[0].append(i)
                    elif m_ind in grps_set[1]:
                        grps_int[1].append(i)
                    else:
                        assert(False)

                # Skip this iteration if the current user has not liked at least (16.66)% men-lead and 16.66% non-male led movies
                if len(grps_int[0]) == 0 or len(grps_int[1]) == 0:
                    continue 
                if len(grps_int[1]) <= 0.2 * len(grps_int[0]) or len(grps_int[0]) <= 0.2 * len(grps_int[1]):
                    continue 

                # Run algorithms and baselines
                sol_baseline_uncons = baseline_uncons(obs_util, grps_int, k, m)
                sol_alg_disj = alg_disj(obs_util, grps_int, k, m) 

                # Record results
                util_baseline_uncons = get_score_rating_overall_users(sol_baseline_uncons, m_inds, m_ind_to_i) / k
                util_alg_disj = get_score_rating_overall_users(sol_alg_disj, m_inds, m_ind_to_i) / k

                list_baseline_uncons.append(util_baseline_uncons) 
                list_alg_disj.append(util_alg_disj) 

            x.append(k)

            LIST_baseline_uncons_OUTER.append(np.mean(list_baseline_uncons))
            LIST_alg_disj_OUTER.append(np.mean(list_alg_disj))

            ERR_LIST_baseline_uncons_OUTER.append(np.std(list_baseline_uncons) / np.sqrt(ITERS))
            ERR_LIST_alg_disj_OUTER.append(np.std(list_alg_disj) / np.sqrt(ITERS))

        x = np.array(x)
        y_uncons = LIST_baseline_uncons_OUTER
        y_disj = LIST_alg_disj_OUTER

        y_uncons_err = ERR_LIST_baseline_uncons_OUTER
        y_disj_err = ERR_LIST_alg_disj_OUTER

        fig, ax = plt.subplots()
        plt.errorbar(x, y_uncons, yerr=y_uncons_err, color="red", label='Uncons',  linewidth=6, alpha=1.0)
        plt.errorbar(x+1, y_disj, yerr=y_disj_err, color="Blue", label='Algorithm 2 (Disjoint)',  linewidth=6, alpha=1.0)


        print(f'y_uncons = {y_uncons}')
        print(f'y_disj = {y_disj}')
        print(f'y_uncons_err = {y_uncons_err}')
        print(f'y_disj_err = {y_disj_err}')


        plt.title(f'ITER={ITERS}, genre={[genres[g] for g in gen]}', fontsize=18)
        plt.ylim(3, 4)
        ax.set_ylabel('$\\frac{1}{k}\\cdot$ (Average User Rating of Recommendation)',fontsize=26)
        ax.set_xlabel('$k$', fontsize=32)
        legend = plt.legend(loc='best', shadow=False, fontsize=26)
        plt.tick_params(axis='both', which='major', labelsize=26)

        # plt.show()
        pdf_savefig()

In [ ]:
run_simulation([1, 2, 6, 18, 19], 1)

In [ ]:
run_simulation([1, 2, 6, 18, 19], 2)

In [ ]:
run_simulation([1, 2, 6, 18, 19], 3)

In [ ]:
run_simulation([1, 2, 6, 18, 19], 4)

In [ ]:
run_simulation([1, 2, 6, 18, 19], 5)